In [16]:
import requests
from dotenv import load_dotenv
import os
import faiss
from sentence_transformers import SentenceTransformer
import torch
from datasets import load_dataset
from sentence_transformers import CrossEncoder
from transformers import BertTokenizer, BertForSequenceClassification
from typing import Literal
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import plotly.express as px
import pandas as pd

load_dotenv()

API_KEY = os.getenv("CLARIN_API_KEY")

MODELS_ENDPOINT = "https://services.clarin-pl.eu/api/v1/oapi/models"
COMPLETIONS_ENDPOINT = "https://services.clarin-pl.eu/api/v1/oapi/chat/completions"

def get_models():
    headers = {
        'Authorization': f'Bearer {API_KEY}',  
        'Content-Type': 'application/json'
    }
    response = requests.get(MODELS_ENDPOINT, headers=headers)  
    if response.status_code == 200:
        return response.json()  

models = get_models()
print(models)


{'data': [{'id': 'bielik', 'full_name': 'speakleash/Bielik-11B-v2.2-Instruct', 'name': 'speakleash/Bielik-11B-v2.2-Instruct'}, {'id': 'cohere', 'full_name': 'CohereForAI/c4ai-command-r-plus', 'name': 'CohereForAI/c4ai-command-r-plus'}, {'id': 'llama3.1-8b', 'full_name': 'meta-llama/Llama-3.1-8B-Instruct', 'name': 'meta-llama/Llama-3.1-8B-Instruct'}, {'id': 'llama', 'full_name': 'meta-llama/Llama-3.1-8B-Instruct', 'name': 'meta-llama/Llama-3.1-8B-Instruct'}, {'id': 'llama-guard', 'full_name': 'meta-llama/Llama-Guard-3-8B', 'name': 'meta-llama/Llama-Guard-3-8B'}, {'id': 'llama3.1', 'full_name': 'meta-llama/Meta-Llama-3.1-70B-Instruct', 'name': 'meta-llama/Meta-Llama-3.1-70B-Instruct'}, {'id': 'openchat', 'full_name': 'openchat/openchat-3.5-1210', 'name': 'openchat/openchat-3.5-1210'}, {'id': 'mixtral-8x22B', 'full_name': 'mistralai/Mixtral-8x22B-Instruct-v0.1', 'name': 'mistralai/Mixtral-8x22B-Instruct-v0.1'}]}


In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = 'cross-encoder/ms-marco-MiniLM-L-6-v2'

In [18]:
class Reranker():
    
    def __init__(self, model_name: str, max_length: int = 512) -> None:
        self.model_name = model_name
        self.max_length = max_length
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        self.model_rank = CrossEncoder(self.model_name, 
                                       max_length = self.max_length, 
                                       device = self.device)
        
        self.tokenizer = BertTokenizer.from_pretrained(self.model_name, 
                                                       max_length = self.max_length)
    
    
    def rank_answers(self, query: str, answers: list[str], top_k: int = 5) -> list[dict[Literal['corpus_id', 'score', 'text'], int | float | str]]:
        self.query = query
        self.top_k = top_k
        self.answers_ranking = self.model_rank.rank(query, 
                                                    answers,
                                                    top_k = 300,
                                                    return_documents = True,
                                                    show_progress_bar = True)
        
        print(f'Zapytanie: {self.query}')
        for k in range(self.top_k):
            print(f'Odpowiedź {k + 1}: {self.answers_ranking[k]["text"]}')
    
    
    def find_embeddings(self) -> list[dict[torch.Tensor, str]]:
        question = self.query
        answers = [dct['text'] for dct in self.answers_ranking]
        texts = [question, *answers]
        labels = ['Zapytanie', *[f'Top {self.top_k} odpowiedzi'] * self.top_k, *['Pozostałe odpowiedzi'] * (len(answers) - self.top_k)]
        tensors = []
        
        for text in texts:
            inputs = self.tokenizer(text, 
                                return_tensors = 'pt', 
                                truncation = True, 
                                padding = True, 
                                max_length = self.max_length).to(device)
            
            with torch.no_grad():
                outputs = self.model_rank.model.bert(**inputs)
                
            embedding = outputs.last_hidden_state[:, 0, :].squeeze()   # EMBEDDING TOKENU [CLS]
            # embedding = outputs.last_hidden_state.mean(dim = 1).squeeze()   # ŚREDNIA Z EMBEDDINGÓW DLA WSZYSTKICH TOKENÓW
            tensors.append(embedding)
        
        emb_tensor = torch.stack(tensors)
        
        self.emb_tensor = emb_tensor
        self.texts = texts
        self.labels = labels
    
    
    def plot_embeddings(self) -> None:
            
        embs, texts, labels  = self.emb_tensor, self.texts, self.labels
        pca = TSNE(n_components = 2, perplexity = 10)
        pca_embs = pca.fit_transform(embs.cpu().numpy())
        
        df = pd.DataFrame({'x': pca_embs[:, 0], 'y': pca_embs[:, 1], 'texts': texts, 'label': labels})
        df['texts'] = df['texts'].apply(lambda text: text[:100] + '...' if len(text) > 100 else text)
        
        
        fig = px.scatter(df, x = 'x', y = 'y', color = 'label',
            title = 'Wizualizacja osadzeń dla modelu re-ranker',
            labels = {'label': 'Legenda'},
            hover_name = df['texts']
        )
        fig.update_traces(marker = dict(size = 10), selector = dict(mode = 'markers'))
        fig.show()

In [6]:
from datasets import load_dataset

dataset = load_dataset("stanfordnlp/imdb")

Generating unsupervised split: 100%|██████████| 50000/50000 [00:00<00:00, 204082.32 examples/s]


In [7]:
def build_faiss_index(dataset, model):
    reviews = dataset["train"]["text"] 
    embeddings = model.encode(reviews, convert_to_numpy=True).astype("float32")

    index = faiss.IndexFlatL2(embeddings.shape[1])  
    index.add(embeddings)  
    return index, reviews


In [ ]:
def search_faiss(query, k=5):
    query_embedding = model.encode([query], convert_to_numpy=True).astype('float32')
    distances, indices = index.search(query_embedding, k)

    result_reviews = [reviews[idx] for idx in indices[0]]
    return distances[0], result_reviews


In [ ]:
def clarin_chat_completion(model_id, prompt, max_tokens=200):
    url = f"{CLARIN_BASE_URL}/chat/completions"
    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}
    data = {
        "model": model_id,
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": max_tokens
    }
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        raise Exception(f"CLARIN API Error: {response.status_code} - {response.text}")

In [23]:
def handle_query_with_rag(user_query, model_id="bielik", k=5):
    distances, result_reviews = search_faiss(user_query, k)

    context = "\n\n".join([f"Recenzja {i+1}: {review}" for i, review in enumerate(result_reviews)])
    if not context.strip():
        context = "Brak dodatkowego kontekstu."

    prompt = f"Zapytanie: {user_query}\n\nKontekst:\n{context}\n\nOdpowiedz w kontekście powyższych informacji."
    response = clarin_chat_completion(model_id, prompt)
    return response


In [13]:
model = SentenceTransformer("all-MiniLM-L6-v2")
index, reviews = build_faiss_index(dataset, model)

In [15]:
faiss.write_index(index, 'index/imdb_faiss.bin')

In [19]:
user_query = "What is the best movie to watch with friends?"
response = handle_query_with_rag(user_query)
print("Odpowiedź:", response)

Odpowiedź: Wybór najlepszego filmu do oglądania z przyjaciółmi zależy od kilku czynników, takich jak gust grupy, okazja oraz tematyka filmu. Ze swojej perspektywy mogę polecić kilka tytułów, które nadają się znakomicie na seans w towarzystwie znajomych.

Jeśli szukasz lekkiego i pełnego humoru filmu, świetną opcją będą komedie romantyczne, takie jak "Notting Hill" czy "Dziewczyna z sąsiedztwa". Oba te filmy nie tylko zapewnią sporo śmiechu, ale także stworzą miłą atmosferę do rozmowy po projekcji.

Dla fanów animacji polecam "Toy Story", serię filmów o Toy Story jest niezwykle uroczym i uniwersalnym wyborem, który spodoba się widzom w każdym wieku. Filmy te mają znakomitą fabułę oraz piękne przesłanie, a ich różnorodność sprawi, że nikt się nie znudzi.

Miłośnicy kina akcji mogą cieszyć się przygodami w cyklu "Mad Max" - te niezwykle dynamiczne i pełne spektakularnych efektów specjalnych produkcje zapewnią mocne wrażenia i dodadzą energii do spotkania ze znajomymi.

A jeśli chcesz wpro

In [24]:
handle_query_with_rag("What do people think about horror movies?")

'W oparciu o podane informacje, nie możemy bezpośrednio określić dokładnej opinii ludzi o horrorach, ponieważ zamiast treści recenzji mamy tylko numery referencyjne. Jednakże na ich podstawie możemy sformułować następujące domysły:\n\n1. Skoro są to recenzje filmów grozy, prawdopodobnie ludzie mają różnorodne opinie - od negatywnych po entuzjastyczne. \n\n2. Numery recenzji sugerują, że opinie byłyby rozłożone na kilka tysięcy widzów (np. dla recenzji 23708 - prawie 24 tysiące). W tak dużej grupie spodziewalibyśmy się również pogłębionej różnorodności preferencji i emocji wobec horrorów.\n\n3. Możliwe, że niektóre z tych recenzji dotyczą klasyków gatunku jak "Psychoza" czy "Piła", inne mogłyby omawiać najnowsze produkcje lub niszowe kino grozy.\n\nAby uzyskać szczegółową opinię publiczną na temat horrorów, potrzebne byłyby pełne treści tych recenzji oraz dodatkowe dane ze sprzedaży biletów, liczby wyświetleń czy reakcji w mediach społecznościowych. Jednocześnie pamiętajmy, że opinie os

______________

In [ ]:
import requests
from dotenv import load_dotenv
import os
import faiss
import torch
from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import pipeline
from datasets import load_dataset

load_dotenv()
API_KEY = os.getenv("CLARIN_API_KEY")


CLARIN_BASE_URL = "https://services.clarin-pl.eu/api/v1/oapi"


def get_models():
    headers = {'Authorization': f'Bearer {API_KEY}', 'Content-Type': 'application/json'}
    response = requests.get(f"{CLARIN_BASE_URL}/models", headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to fetch models: {response.status_code} - {response.text}")

dataset = load_dataset("stanfordnlp/imdb")

def build_faiss_index(dataset, model):
    reviews = dataset["train"]["text"]
    embeddings = model.encode(reviews, convert_to_numpy=True).astype("float32")
    
    d = embeddings.shape[1]  
    quantizer = faiss.IndexFlatL2(d)  

    nlist = 100 
    m = 8  
    nbits = 8  
    index = faiss.IndexIVFPQ(quantizer, d, nlist, m, nbits)  

    index.train(embeddings)  
    index.add(embeddings)  

    faiss.write_index(index, "index/imdb_faiss_ivfpq.bin")
    print("Indeks został zapisany na dysk.")

    return index, reviews


def search_faiss(index, query, model, reviews, k=5):
    query_embedding = model.encode([query], convert_to_numpy=True).astype("float32")
    distances, indices = index.search(query_embedding, k)
    result_reviews = [reviews[idx] for idx in indices[0]]
    return distances[0], result_reviews

def rerank_with_cross_encoder(query, result_reviews, reranker, k=5):

    pairs = [[query, review] for review in result_reviews]

    rerank_scores = reranker.predict(pairs)

    ranked_reviews = [result_reviews[i] for i in sorted(range(len(rerank_scores)), key=lambda i: rerank_scores[i], reverse=True)]
    return ranked_reviews[:k]

def clarin_chat_completion(model_id, prompt, max_tokens=300):
    url = f"{CLARIN_BASE_URL}/chat/completions"
    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}
    data = {
        "model": model_id,
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": max_tokens
    }
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        raise Exception(f"CLARIN API Error: {response.status_code} - {response.text}")

model = SentenceTransformer("all-MiniLM-L6-v2")

index, reviews = build_faiss_index(dataset, model)


Indeks został zapisany na dysk.


In [ ]:
def handle_query_with_rag(user_query,reranker, model_id="mixtral-8x22B", k=8):
    distances, result_reviews = search_faiss(index, user_query, model, reviews, 50)

    reranked_reviews = rerank_with_cross_encoder(user_query, result_reviews, reranker, k)

    context = "\n\n".join([f"Recenzja {i+1}: {review}" for i, review in enumerate(reranked_reviews)])
    if not context.strip():
        context = "Brak dodatkowego kontekstu."

    prompt = f"Zapytanie: {user_query}\n\nKontekst:\n{context}\n\nOdpowiedz zwięźle w kontekście powyższych informacji."
    response = clarin_chat_completion(model_id, prompt)
    return response, context

def evaluate_response_with_llama(question, context, answer, model_id="bielik"):

    prompt = f"Na podstawie poniższego kontekstu i pytania, oceń, jak trafna i spójna jest odpowiedź modelu '{answer}' w odniesieniu do pytania '{question}':\n\nKontekst: {context}\n\nCzy odpowiedź jest trafna i zgodna z kontekstem? Oceń odpowiedź na skali od 1 do 10, gdzie 1 to 'bardzo nietrafna', a 10 to 'bardzo trafna'. Dodaj krótki komentarz co poprawić."

    evaluation = clarin_chat_completion(model_id, prompt)
    return evaluation

In [ ]:
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

user_query = "What are the worst movies?"

response, context = handle_query_with_rag(user_query, reranker)

print("Użyty kontekst:\n", context)

evaluation = evaluate_response_with_llama(user_query, context, response)

print("Odpowiedź:", response)
print("Ocena:", evaluation)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Użyty kontekst:
 Recenzja 1: This is hands down the worst movie of all time. A combination of Whoopie Goldberg (the worst actress/person in history) and a talking dinosaur ala Jar-Jar-Binks add up to a painfully bad movie. That was an understatement. This movie is unwatchable. For the love of God, do not watch this movie.

Recenzja 2: The thing about calling "House of the Dead" the worst movie of all time is that it's really not. There are worse movies out there. I watch alot of Hong Kong ninja movies that are basically the result of an unfinished Japanese police drama having footage of ninjas inserted at the end to create something that could technically be called "a movie."<br /><br />House of the Dead is however one of the worst films I've ever seen at the theatres. Walking out half way through, I actually felt I was somewhat dumber for having set through 45 minutes of this piece of garbage.

Recenzja 3: This is surely one of the worst films ever made. Each scene is painful. You wil

___________________

In [1]:
import faiss
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, CrossEncoder
from datasets import load_dataset
import requests
from dotenv import load_dotenv
import os

# Wczytanie danych
dataset = load_dataset("stanfordnlp/imdb")
reviews = list(set(dataset['train']["text"]))

model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')
device = torch.device("cuda")
model.to(device)

# Tworzenie osadzeń dla recenzji
#reviews_embeddings = model.encode(reviews, convert_to_tensor=True)

reviews_embeddings = np.load("reviews_embeddings.npy")

In [2]:
len(reviews_embeddings)

24904

In [ ]:
# from tqdm import tqdm
# import numpy as np
# import faiss

# # Example: Split the embeddings into batches and add them with tqdm for a progress bar
# reviews_embeddings = np.load("reviews_embeddings.npy")
# batch_size = 1000  # Set the batch size
# embeddings_np = reviews_embeddings

# # Initialize the FAISS index (assuming you already have your index setup)
# res = faiss.StandardGpuResources()
# d = embeddings_np.shape[1]
# index = faiss.IndexHNSWFlat(d, 32)
# index = faiss.index_cpu_to_gpu(res, 0, index)

# # Use tqdm to show progress while adding in batches
# for i in tqdm(range(0, len(embeddings_np), batch_size), desc="Adding embeddings"):
#     batch = embeddings_np[i:i + batch_size]
#     index.add(np.ascontiguousarray(batch))


Adding embeddings:  20%|██        | 5/25 [23:58<2:03:48, 371.42s/it]

In [2]:
# Zapisz indeks do pliku
index = faiss.read_index(r"C:\Users\kjani\nlp\NLP\task_3\index_faiss.index")

In [ ]:
def search_faiss(index, query, model, reviews, k=5):
    query_embedding = model.encode(query, convert_to_tensor=True).detach().cpu().numpy().reshape(1, -1)
    distances, indices = index.search(query_embedding, k)
    return distances[0], [reviews[i] for i in indices[0]]

def rerank_with_cross_encoder(question, retrieved_reviews, reranker, k=5, return_indexes=False):
    results = reranker.rank(question, retrieved_reviews, top_k = k, return_documents = not return_indexes, show_progress_bar = True)

    if return_indexes: return [result['corpus_id'] for result in results]

    return [result['text'] for result in results]

model_rank = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', max_length=512, device='cuda')

load_dotenv()
API_KEY = os.getenv("CLARIN_API_KEY")
COMPLETIONS_ENDPOINT = "https://services.clarin-pl.eu/api/v1/oapi/chat/completions"

def clarin_chat_completion(model_id, prompt, max_tokens=350):
    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}
    data = {
        "model": model_id,
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": max_tokens
    }
    response = requests.post(COMPLETIONS_ENDPOINT, json=data, headers=headers)
    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        raise Exception(f"CLARIN API Error: {response.status_code} - {response.text}")

def handle_query_with_rag(user_query, reranker, model_id="mixtral-8x22B", k=8):
    distances, result_reviews = search_faiss(index, user_query, model, reviews, 200)
    reranked_reviews = rerank_with_cross_encoder(user_query, result_reviews, reranker, k)
    context = "\n\n".join([f"Recenzja {i+1}: {review}" for i, review in enumerate(reranked_reviews)])
    if not context.strip():
        context = "Brak dodatkowego kontekstu."
    prompt = f"""
Kontekst:
{context}

ZAPYTANIE:
{user_query}

INSTRUKCJE:
Odpowiedz na zapytanie użytkownika, korzystając tylko z kontekstu.

Zachowaj odpowiedź zwięzłą i trafną w kontekście pytania. Używaj argumentów opartych na faktach z kontekstu, unikaj ogólnych spekulacji. Jeśli to możliwe, odwołaj się do kilku fragmentów dokumentu.

Nie odbiegaj od faktów zawartych w kotekście.

Odpowiedź napisz w języku polskim.
"""
    print(context)
    response = clarin_chat_completion(model_id, prompt)
    return response, context

def evaluate_response_with_llama(question, context, answer, model_id="bielik"):
    prompt = f"Na podstawie poniższego kontekstu i pytania, oceń, jak trafna i spójna jest odpowiedź modelu '{answer}' w odniesieniu do pytania '{question}':\n\nKontekst: {context}\n\nCzy odpowiedź jest trafna i zgodna z kontekstem? Oceń odpowiedź na skali od 1 do 10, gdzie 1 to 'bardzo nietrafna', a 10 to 'bardzo trafna'. Dodaj krótki komentarz, co model mógłby poprawić."
    evaluation = clarin_chat_completion(model_id, prompt)
    return evaluation


In [11]:
user_query = "What movie I can watch with friends?"
response, context = handle_query_with_rag(user_query, model_rank, model_id="mixtral-8x22B")
print("Odpowiedź:", response)

evaluation = evaluate_response_with_llama(user_query, context, response, model_id="bielik")
print("Ocena odpowiedzi:", evaluation)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Recenzja 1: This little-appreciated movie is one of my favorites. I can watch it over and over. Dreyfus and Braga are masterful, but Raul Julia steals the show! A tongue-in-cheek, menacingly humorous Gomez Addams, with just the right tone for this irreverent spoof of this oft-told story.<br /><br />Generally untrumpeted and unappreciated, Moon Over Parador allows you to check out of reality and join the fun going on up on the screen. Two thumbs up!

Recenzja 2: ...however I am not one of them. Caro Diario at least was watchable for two thirds of the time, but the boring and self-centred third section of that movie gave us a taste of what was to come in this extraordinarily self-indulgent mess. Moretti says he feels a need to make this movie, but doesn't want to, whereas the viewer feels that he should stick with it, but really doesn't want to either. A film about Italian politics and elections could be fascinating, but this is not that film. At one point, Moretti and his friends are st

In [18]:
user_query = "What are the best action movies to watch?"
response, context = handle_query_with_rag(user_query, model_rank, model_id="mixtral-8x22B", k=10)
print("Odpowiedź:", response)

evaluation = evaluate_response_with_llama(user_query, context, response, model_id="bielik")
print("Ocena odpowiedzi:", evaluation)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Recenzja 1: The van trotta movie rosenstrasse is the best movie i have seen in years. i am actually not really interested in films with historical background but with this she won my interest for that time!!<br /><br />the only annoying thing about the movie have been the scenes in new york, and the impression i had of "trying to be as American as possible" ... which i think has absolutely failed.<br /><br />the scenes in the back really got to my heart. the German actress katja riemann completely deserved her award. she is one of the most impressing actress i have ever seen. in future i will watch more of her movies. great luck for me that i am a native German speaking =) and only for a year in the us, so as soon as i am back i'll buy some riemann dvds.<br /><br />so to all out there who have not seen this movie yet: WATCH IT!!! i think it would be too long to describe what it is all about yet, especially all the flash backs and switches of times are hard to explain, but simply watcxh

In [13]:
user_query = "Which movies have the most emotional stories?"
response, context = handle_query_with_rag(user_query, model_rank, model_id="mixtral-8x22B", k=10)
print("Odpowiedź:", response)

evaluation = evaluate_response_with_llama(user_query, context, response, model_id="bielik")
print("Ocena odpowiedzi:", evaluation)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Recenzja 1: I came here for a review last night before deciding which TV movie to settle in front of, and those I found made this one look unmissable. How misled I feel!<br /><br />Firstly, it needs to be pointed out up front that this is very much a housewife's daytime movie. The performances are wooden, every sentence is an attempt at 'poignant' in the way that housewife's daytime movies and bad soap operas always are, and it is based in that predictable and well-trodden premise that men (particularly soldiers) are essentially violent and incompassionate. The whole movie is about the 'drama' apparent in the moments when the male characters threaten to develop a second dimension.<br /><br />If that sounds tolerable (or even enjoyable) to you, then be warned. Linda Hamilton's German accent, while quite good, is painfully distracting - as is her face, for some reason. The other performances are no doubt an enduring source of embarrassment to their perpetrators, with painfully thin and o

In [14]:
user_query = "What are the best horror movies?"
response, context = handle_query_with_rag(user_query, model_rank, model_id="mixtral-8x22B", k=10)
print("Odpowiedź:", response)

evaluation = evaluate_response_with_llama(user_query, context, response, model_id="bielik")
print("Ocena odpowiedzi:", evaluation)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Recenzja 1: Tobe Hooper is quite possibly the biggest fluke the horror genre has to offer. Like any other horror fan, I loved the Texas Chainsaw, but I think that in order to put your name in front on a movie title, you should have at least more than one hit movie. I can't really think of any other movie Hooper has done (on his own, don't count Poltergeist) that has really made an impact on the horror genre or film world. And this movie, Night Terrors, just backs up my point.<br /><br />Poor Robert Englund, I give him credit for at least doing a good job with the awful material he was given. He did what he could. As for the movie itself? Pure drudge. Unnecessary nude scenes every five minutes, a story that must have been penned in an our, and really just awful scenery, music, and cinematography. Nothing in this film is redeemable. Don't waste your time.<br /><br />Overall, 1 out of 10. I feel sorry for Hooper, his career seems like it was over before it really ever got started. I hope 

In [15]:
user_query = "Which movie has the best soundtrack?"
response, context = handle_query_with_rag(user_query, model_rank, model_id="mixtral-8x22B", k=10)
print("Odpowiedź:", response)

evaluation = evaluate_response_with_llama(user_query, context, response, model_id="bielik")
print("Ocena odpowiedzi:", evaluation)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Recenzja 1: "Foxes" is one movie I remember for it's portrayal of teenagers in the late 1970's. As I am exactly Jodie Foster's age, I related to this movie. It deals with the frustrations, temptations, relationships & rebellion of youth. The soundtrack is great with inspiring rock eg. "More Than a Feeling" by Boston and sad numbers like "On the Radio" by Donna Summer. The music of my late teens. Yep, I'll always remember this one, even if it wasn't huge.

Recenzja 2: Besides the fact that my list of favorite movie makers is: 1)Stanley Kubrick 2)God Allmighty 3)the rest... this movie actually is better than the book (and the TV miniseries though this is an easy feat, considering the director). The flawless filming stile, the acting and (Kubrick's all time number one skill) the music - make it THE masterpiece of horror. I watched the TV miniseries a few years ago and liked the story and I had my hopes about this when I got a hold of it. IT BLEW ME AWAY!!! It is far better than I ever ima

In [16]:
user_query = "Can you recommend some great movies for fans of psychological thrillers?"
response, context = handle_query_with_rag(user_query, model_rank, model_id="mixtral-8x22B", k=10)
print("Odpowiedź:", response)

evaluation = evaluate_response_with_llama(user_query, context, response, model_id="bielik")
print("Ocena odpowiedzi:", evaluation)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Recenzja 1: And this somebody is me. And not only me, as I can see here at IMDb or when leaving the theater. Why did the people love it? It's obvious: Everybody knows zombies by now (at least the Horror fans by heart and the others through the "Dawn of the Dead" reinvention or Resident Evil movies etc.) <br /><br />Or at least they thought they knew everything about zombies ... that is until this movie came along. And you'll see zombies in a new light (perhaps). This is not a horror movie, although it does contain some violent scenes, but is rather a comedy. A satire to be precise. And it never runs out of steam! That is why I rated it so high. Pacing wise it's incredible, the acting is great and the script has no (obvious) mistakes ... quite the contrary: It's a gem and if you're only a little bit interested in zombies you ought to see it! And even if you dislike them, watch it! Because it's a great (comedy) movie!

Recenzja 2: When I found the movie in the schedule for Christmas, its

In [17]:
user_query = "What are the movies with best endings?"
response, context = handle_query_with_rag(user_query, model_rank, model_id="mixtral-8x22B", k=10)
print("Odpowiedź:", response)

evaluation = evaluate_response_with_llama(user_query, context, response, model_id="bielik")
print("Ocena odpowiedzi:", evaluation)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Recenzja 1: As we all know a romantic comedy is the genre with the ending already known. The two leads always have to get together. Late in the third act I was trying to figure out how this will wrap up and how they will end up together. A clue was given right from the start, but you'll never realize it until the end. It's a simple hook, but it works. It Had To Be You cover a lot of the usual ground, but takes a fresh spin when ever possible. I liked all the NY characters and I loved the locations. It's a postcard of NY. Also it was nice to watch a film and not find anything offensive in it. So, if you like a good old fashion romantic movie ... then this is for you.

Recenzja 2: Finally a thriller which omits the car chases, explosions and other eye catching effects. The movie combines a simple plot (assasination of a french president) with an excellent background. It takes a look behind mans behavior with authorities, and explains why we would obey almost every order (even murder) whi

In [23]:
user_query = "Wchich movie is the lost of time?"
response, context = handle_query_with_rag(user_query, model_rank, model_id="mixtral-8x22B", k=10)
print("Odpowiedź:", response)

evaluation = evaluate_response_with_llama(user_query, context, response, model_id="bielik")
print("Ocena odpowiedzi:", evaluation)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Recenzja 1: seriously what the hell was this movie about,,simply stupid,,i'd give it 0 but,,,1"awful" is the lowest you can go,,seriously this movie is not worth watching,,waste of time, i don't know what the hell is wrong with you guys voting this movie 7 out of 10,,i seriously can make a better movie than this , hire some other unemployed people,,'n i promise i'll make a movie better than this,,this movie was so bad,,that i'll never watch a movie starring Steve Carrel again,bottom line don't waste your time to download it off the net or rent it,,i'd nominate this movie for the worst movie of the century i mean the worst is Something Gotta give but after that this is the second

Recenzja 2: This is the biggest pile of crap I have ever watched. DO NOT RENT! The makers of this movie should be band from ever making another movie. It starts with some what of a plot, then fades fast to nothing. I think I would rather watch paint dry then to as much as looking at the cover. The actors were 